<a href="https://colab.research.google.com/github/amal-ro-shan/Info-aidTech/blob/main/task_2_Movie_Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156224 sha256=6e0b7be2d3a6a006a640f1235dffe0fc73af215f8cbef14dfb64456a0a56c600
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

Loading Dataset

In [3]:
m = pd.read_csv("/content/movies.csv")
m

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


In [4]:
m.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Preprocessing Dataset

In [5]:
m.drop_duplicates(inplace= True)

In [6]:
m.dropna(inplace=True)

In [7]:
mat = m.pivot(index='title', columns='movieId', values='genres').fillna(0)
print(mat)


movieId                                       1      2      3      4       \
title                                                                       
'71 (2014)                                         0      0      0      0   
'Hellboy': The Seeds of Creation (2004)            0      0      0      0   
'Round Midnight (1986)                             0      0      0      0   
'Til There Was You (1997)                          0      0      0      0   
'burbs, The (1989)                                 0      0      0      0   
...                                              ...    ...    ...    ...   
loudQUIETloud: A Film About the Pixies (2006)      0      0      0      0   
xXx (2002)                                         0      0      0      0   
xXx: State of the Union (2005)                     0      0      0      0   
¡Three Amigos! (1986)                              0      0      0      0   
À nous la liberté (Freedom for Us) (1931)          0      0      0      0   

Splitting the dataset

In [10]:
reader = Reader(rating_scale=(0.5, 5))
d = Dataset.load_from_df(m[['genres', 'title', 'movieId']], reader)
trainset, testset = train_test_split(d, test_size=0.2, random_state=42)

Implementing collabrative filtering


In [11]:
algo = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
algo.fit(trainset)

Perfomance evaluation

In [12]:
predictions = algo.test(testset)
r = accuracy.rmse(predictions)

RMSE: 49981.2745


Movie Recommendation

In [13]:
user_id = 50
user_data = m[m['movieId'] == user_id]
user_rated_movie_ids = set(user_data['movieId'])

In [21]:
all_movie_ids = set(m['movieId'])
unrated_movie_ids = all_movie_ids - user_rated_movie_ids
unrated_movie_predictions = [(user_id, movie_id, algo.predict(user_id, movie_id).est) for movie_id in unrated_movie_ids]

In [23]:
N = 6
top_n_recommendations = sorted(unrated_movie_predictions, key=lambda x: x[2], reverse=True)[:N]

In [25]:
print("Top Movie Recommendations:")
for _, movie_id, predicted_rating in top_n_recommendations:
    movie_title = m[m['movieId'] == movie_id]['title'].values[0]
    print(f"Movie: {movie_title}, Predicted Rating: {predicted_rating}")

Top Movie Recommendations:
Movie: So Big! (1932), Predicted Rating: 5
Movie: Toy Story (1995), Predicted Rating: 5
Movie: Jumanji (1995), Predicted Rating: 5
Movie: Grumpier Old Men (1995), Predicted Rating: 5
Movie: Waiting to Exhale (1995), Predicted Rating: 5
Movie: Father of the Bride Part II (1995), Predicted Rating: 5


New User Rating

In [26]:
new_user_ratings = {
    'So Big! (1932)': 4.3,
    'Jumanji (1995)': 3.6,
    'Heat (1995)': 4.6
}

In [27]:
unrated_movie_predictions = [
    (user_id, movie_id, algo.predict(user_id, movie_id).est)
    for movie_id in unrated_movie_ids
]


In [28]:
N = 6
top_n_recommendations = sorted(unrated_movie_predictions, key=lambda x: x[2], reverse=True)[:N]
